In [9]:
import pandas as pd
from pymongo import MongoClient

client = MongoClient('mongodb://localhost:27017/')
db = client.companies

In [18]:
#Starting consuting time on Mongo DB

one_office = db.companies.find({    #Companies with at least 1 office
    "offices":{
        "$not":{
            "$size":0
        }
    }
},{"name":1, "offices":1})

one_office_df = pd.DataFrame(one_office)
#display(one_office_df)
print(one_office_df.shape)
one_office_df.columns


(13744, 3)


Index(['_id', 'name', 'offices'], dtype='object')

In [ ]:
#Filtering with mongo

{    "$and":[
        {"offices.latitude": {"$exists": True}},
        {"offices.longitude": {"$exists": True}},
        {"founded_year": {"$exists": True}}, 
        {"founded_year": {"$gte": 2009}},
        {"category_code": {"$exists": True}},
        {"total_money_raised": {"$exists": True}},
        {"total_money_raised": { "$and": [ 
                                    { "$gte": 1000000 }, 
                                    { <expression2> }, 
                                    { "tags": { "$in": ["K", "M"] } },
                                ]
                            },
        {"$or": [                              {"category_code":"games_video"},
                                               {"category_code":"software"},
                                               {"category_code":"web"},
                                               {"category_code":"mobile"},
                                               {"category_code":"photo_video"},
                                               {"category_code":"ecommerce"},
                                               {"category_code":"search"},
                                               {"category_code":"network_hosting"}
{"_id": 0, "category_code": 1,"founded_year": 1, "name": 1, 
    "number_of_employees": 1, "offices.country_code": 1, 
    "offices.latitude": 1, "offices.longitude": 1, "total_money_raised": 1})


{$and: [ {number_of_employees: { $lte: 1000 }}, {founded_year: { $lte: 2005 }}]}




db.companies.find({    #Companies with at least 1 office
    "offices":{
        "$not":{
            "$size":0
        }
    }
},{"name":1, "offices":1}

In [ ]:
({"$and": [{"offices.latitude": {"$exists": True}}, 
           {"offices.latitude": {"$ne": None}},
           {"offices.longitude": {"$exists": True}}, 
           {"offices.longitude": {"$ne": None}},
           {"category_code": {"$exists": True}}, 
           {"category_code": {"$ne": None}},
           {"number_of_employees": {"$exists": True}}, 
           {"number_of_employees": {"$ne": None}},
           {"founded_year": {"$exists": True}}, 
           {"founded_year": {"$gte": 2008}},
           {"deadpooled_year": None},
           {"category_code": {"$ne": "other"}}, 
           {"category_code": {"$ne": "semiconductor"}}, 
           {"category_code": {"$ne": "finance"}}, 
           {"category_code": {"$ne": "medical"}}, 
           {"category_code": {"$ne": "health"}}, 
           {"category_code": {"$ne": "manufacturing"}}, 
           {"category_code": {"$ne": "real_estate"}}, 
           {"category_code": {"$ne": "fashion"}}, 
           {"category_code": {"$ne": "nanotech"}}, 
           {"category_code": {"$ne": "hospitality"}}, 
           {"category_code": {"$ne": "transportation"}}, 
           {"category_code": {"$ne": "local"}}]}, 
                 {"_id": 0, "category_code": 1,"founded_year": 1, "name": 1, 
                    "number_of_employees": 1, "offices.country_code": 1, 
                    "offices.latitude": 1, "offices.longitude": 1, "total_money_raised": 1})

In [20]:
def getFirstOffice(data):
    data = data['offices']
    #Only create the geoJSON object if all geodata is available
    principal = None
    if data[0]['latitude'] and data[0]['longitude']:
        principal = {
            "type":"Point",
            "coordinates":[data[0]['longitude'], data[0]['latitude']]
        }
    return {
        "totalOffices": len(data),
        "lat": data[0]['latitude'],
        "lng": data[0]['longitude'],
        "oficina_principal": principal
    }


first_office = one_office_df[["offices"]].apply(getFirstOffice, result_type = "expand", axis = 1)
print(first_office.shape)
display(first_office.head())

(13744, 4)


,lat,lng,oficina_principal,totalOffices
0,37.692934,-121.904945,"{'type': 'Point', 'coordinates': [-121.904945,...",1.0
1,47.603122,-122.333253,"{'type': 'Point', 'coordinates': [-122.333253,...",2.0
2,37.692934,-121.904945,"{'type': 'Point', 'coordinates': [-121.904945,...",1.0
3,NaN,NaN,NaN,1.0
4,34.090368,-118.393064,"{'type': 'Point', 'coordinates': [-118.393064,...",1.0


In [23]:
df_clean = pd.concat([one_office_df,first_office], axis=1)[["name","lat","lng", "oficina_principal","totalOffices"]]

display(df_clean.head())


,name,lat,lng,oficina_principal,totalOffices
0,AdventNet,37.692934,-121.904945,"{'type': 'Point', 'coordinates': [-121.904945,...",1.0
1,Wetpaint,47.603122,-122.333253,"{'type': 'Point', 'coordinates': [-122.333253,...",2.0
2,Zoho,37.692934,-121.904945,"{'type': 'Point', 'coordinates': [-121.904945,...",1.0
3,Omnidrive,NaN,NaN,NaN,1.0
4,Geni,34.090368,-118.393064,"{'type': 'Point', 'coordinates': [-118.393064,...",1.0


In [50]:
def drop_null(df):
    return df.dropna(axis=0)

In [54]:
df_cleaned = df_clean.dropna()

In [56]:
df_cleaned.head()
df_cleaned.shape


(9618, 5)